<a href="https://colab.research.google.com/github/suhbaca/AI4GOOD_TeamProject/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **LLAMA 2 Model**



